# mt5-thai-QG
This is notebook detailing how to finetune **mt5 for question-generation in the Thai language** 
## Installing Requirements and Clone Git Repo

In [ ]:
# Remove "sample_data" in colab
!rm -rf *
!git clone https://github.com/parinzee/QuestionGenerationT5.git ./
%pip install ijson pandas simplet5 beautifulsoup4

## Gather & Process datasets
- xquad-thai
- iapp-wiki-qa-dataset
- thaiqa

In [1]:

import math
import pandas as pd
import urllib.request
import os
import ijson
import json
import re
from zipfile import ZipFile
from bs4 import BeautifulSoup


def download_dataset(url, file_name):
    urllib.request.urlretrieve(
        url,
        os.path.join("dataset/", file_name),
        reporthook=(
            lambda count, block, total: print(
                f"Downloading {file_name}: {math.floor((count * block) / total * 100)}%",
                end="\r",
            )
        ),
    )
    print(f"Downloaded {file_name} from {url}")


# Check if the dataset already exists
if not (os.path.exists("dataset/xquad.json") and os.path.exists("dataset/iapp-thai-wikipedia-qa.json")):

    # Download all datasets
    download_dataset("https://github.com/deepmind/xquad/raw/master/xquad.th.json", "xquad.json")
    download_dataset("https://raw.githubusercontent.com/iapp-technology/iapp-wiki-qa-dataset/main/iapp-thai-wikipedia-qa-1961-docs-9170-questions.json", "iapp-thai-wikipedia-qa.json")
    download_dataset("https://github.com/PyThaiNLP/thaiqa_squad/raw/main/data.zip", "thaiqa.zip")
    with ZipFile("dataset/thaiqa.zip") as zipfile:
        os.mkdir("dataset/thaiqa")
        zipfile.extractall("dataset/thaiqa/")

# This list will store all the Q&A
source_list = []
target_list = []

# Start cleaning data
squad = open(os.path.join("dataset/", "xquad.json"))
iapp = open(os.path.join("dataset/", "iapp-thai-wikipedia-qa.json"))
iapp_keys = open(os.path.join("dataset/", "iapp-thai-wikipedia-qa.json"))
thaiqa = open(os.path.join("dataset/thaiqa/data/train.jsonl"))

squad_json = ijson.items(squad, "data.item")
iapp_json = json.load(iapp)
iapp_keys = ijson.kvitems(iapp_keys, "db")
thaiqa_df = pd.read_json(thaiqa, lines=True)

# Get data from xquad
for obj in squad_json:
    paragraphs = obj["paragraphs"]
    for p in paragraphs:
        context = p["context"]
        qas = [p for p in p["qas"] if len(p) > 0]

        source_text = f"สร้าง {len(qas)} คำถาม: {context}"
        target_text = ""

        for number, qa in enumerate(qas):
            target_text += (
                f"{number + 1}. {qa['question']} A: {qa['answers'][0]['text']} "
            )

        source_list.append(source_text.strip())
        target_list.append(target_text.strip())

# Get dataset from iapp
for key in iapp_keys:
    try:
        obj = iapp_json["db"][key[0]] 
        context = obj["detail"]
        qas = obj["QA"]
        target_text = ""

        qa_amount = 0

        for number, qa in enumerate(qas):
            if len(qa["a"]) != 0 and len(qa["q"]) != 0:
                target_text += (
                    f"{number + 1}. {qa['q']} A: {qa['a'][0]} "
                )
                qa_amount += 1

        source_text = f"สร้าง {qa_amount} คำถาม: {context}"
        source_list.append(source_text.strip())
        target_list.append(target_text.strip())
    
    except KeyError as e:
        print(f"Error: {e} (Error on 'detail' is expected because of the dataset structure")
    
# Get data from thaiqa
article_ids = set(thaiqa_df["article_id"])
for id in article_ids:
    questions = thaiqa_df[thaiqa_df["article_id"] == id]

    # Remove html markup
    soup = BeautifulSoup(questions["context"].iloc[0])

    # Remove parenthesis because some are empty
    context = re.sub(r'\(\)', '', soup.text)

    # Remove double spaces resulting from removing parenthesis
    context = re.sub(r'\s\s+', " ", context)

    source_text = f"สร้าง {len(questions)} คำถาม: {context}"
    target_text = ""

    qa_number = 1
    for _, question in questions.iterrows():
        target_text += f"{qa_number}. {question['question']} A: {question['answer']} "
        qa_number += 1
    
    source_list.append(source_text.strip())
    target_list.append(target_text.strip())

dataframe = pd.DataFrame({"source_text": source_list, "target_text": target_list})

train_df = dataframe.sample(frac=0.85, random_state=20)
test_df = dataframe.drop(train_df.index)

Error: 'detail' (Error on 'detail' is expected because of the dataset structure
1. ชื่อย่อของมหาวิทยาลัยโรเจอร์ วิลเลียมส์ ใช้ว่าอะไร A: RWU 2. มหาวิทยาลัยโรเจอร์ วิลเลียมส์ เป็นมหาวิทยาลัยทางด้านศิลปศาสตร์ของอเมริกา ตั้งอยู่ในรัฐใด A: รัฐโรดไอแลนด์ 
1. ใครเป็นผู้แต่งหนังสือนิทานเด็กเรื่องนิทานของบีเดิลยอดกวี A: เจ. เค. โรว์ลิ่ง 
1. ผู้เขียนบทละครชุดของไทยเรื่อง ฮอร์โมนส์ วัยว้าวุ่น คือใคร A: ทรงยศ สุขมากอนันต์ 
1. วรานิษฐ์ จิราโรจน์เจริญ เข้าวงการบันเทิงจากการถ่ายมิวสิควีดีโอของนักร้องคนใด A: ทาทา ยัง 
1. ราอุล อัลบิโอล ตอร์ตาฆาดา เล่นฟุตบอลให้กับทีมชาติอะไร A: ทีมชาติสเปน 2. ราอุล อัลบิโอล ตอร์ตาฆาดาเป็นนักฟุตบอลชาวสเปน เกิดเมื่อวันที่เท่าไร A: 4 
1. สสารชนิดใดที่มีคุณสมบัติกัดกร่อนโลหะ หินปูน หรือเนื้อเยื่อของร่างกาย A: กรด 2. กรดอนินทรีย์คืออะไร A: กรดที่ได้จากแร่ธาตุ 
1. สัญญาณไฟจราจรแห่งแรกของโลกติดตั้งขึ้นเมื่อปีค.ศ.ใด A: 1868 2. สัญญาณไฟจราจรแห่งแรกของโลกติดตั้งขึ้นที่เมืองใด A: ลอนดอน 
1. จักรพรรดิเค็นโซ เป็นจักรพรรดิองค์ที่ 23 แห่งราชวงศ์ใด A: ญี่ปุ่น 
1. นางสาวอรอนงค์ กาญจนช

## Training

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="mt5", model_name="google/mt5-small")

model.train(train_df=train_df,
          eval_df=test_df, 
          source_max_token_len=1024, 
          target_max_token_len=1024, 
          batch_size=1, max_epochs=20, use_gpu=True)

## Load trained model

In [ ]:
# let's load the trained model for inferencing:
model.load_model("mt5","", use_gpu=True)

text_to_summarize="""สร้าง 3 คำถาม: ดันเจียนซีจ (อังกฤษ: Dungeon Siege) เป็นเกมแอ็กชันเล่นตามบทบาทที่พัฒนาโดยแก๊สเพาเวิร์ดเกมส์ ซึ่งไมโครซอฟท์ได้จัดจำหน่ายบนแพลตฟอร์มไมโครซอฟท์ วินโดวส์ ในเดือนเมษายน ค.ศ. 2002 และเดสทิเนียร์ได้จัดจำหน่ายบนแพลตฟอร์มแมคโอเอสเท็นในปีถัดไป โดยมีฉากอยู่ในอาณาจักรยุคกลางสมมติ ชื่อ เอห์บ เกมนี้ยังจัดเป็นแนวแฟนตาซีระดับสูงที่เดินเรื่องตามชาวไร่หนุ่มคนหนึ่งและเพื่อนร่วมทางขณะที่พวกเขาออกเดินทางเพื่อกำจัดกองกำลังที่รุกราน ในตอนแรก กลุ่มตัวเอกเพียงต้องการเตือนเมืองใกล้เคียงเกี่ยวกับการรุกรานของเผ่าพันธุ์สิ่งมีชีวิตที่ชื่อครุก และในอีกไม่นาน ชาวไร่คนดังกล่าวและเพื่อนร่วมทางกับเขาตกอยู่ในสถานการณ์หาทางเอาชนะเผ่าพันธุ์อื่นที่เรียกว่าเซกอย่างหลีกเลี่ยงไม่ได้ ซึ่งฟื้นคืนพลังใหม่หลังจากถูกคุมขังอยู่ 300 ปี โลกของดันเจียนซีจไม่ใช้ระบบเลเวลเหมือนกับวิดีโอเกมเล่นตามบทบาทอื่น ๆ ในยุคนั้น หากแต่เป็นพื้นที่เดียวที่ต่อเนื่อง โดยปราศจากการโหลดหน้าจอ ซึ่งผู้เล่นเดินทางผ่านเพื่อต่อสู้กับฝูงศัตรู นอกจากนี้ แทนที่จะกำหนดคลาสตัวละครและควบคุมตัวละครทั้งหมดในกลุ่มด้วยตนเอง ผู้เล่นจะควบคุมกลยุทธ์และอาวุธ ตลอดจนการใช้เวทมนตร์โดยรวมของพวกเขา ซึ่งกำกับการเติบโตของตัวละคร
"""

print(model.predict(text_to_summarize, max_length=1024)[0])